In [2]:
import requests
import json

# Define the OpenDota API base URL
OPENDOTA_API_URL = "https://api.opendota.com/api/matches/"

# Function to get match data from OpenDota
def get_match_data(match_id):
    url = f"{OPENDOTA_API_URL}{match_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching match data:", response.status_code)
        return None

# Function to print team fights with start, peak (highest damage), and end times
def print_team_fights(teamfights):
    print("\n--- Team Fights ---")
    for i, fight in enumerate(teamfights):
        start = fight['start']
        end = fight['end']
        
        # Filter out players without 'damage_done' to avoid KeyError
        players_with_damage = [p for p in fight['players'] if 'damage_done' in p]
        
        # Check if there are players with damage data
        if players_with_damage:
            peak_damage = max(players_with_damage, key=lambda x: x['damage_done'])['damage_done']
        else:
            peak_damage = "No damage data available"
        
        print(f"Team Fight {i + 1} - Start: {start}, Peak Damage: {peak_damage}, End: {end}")


# Function to print objectives like Roshan, towers, and barracks
def print_objectives(objectives):
    print("\n--- Objectives ---")
    for obj in objectives:
        type_ = obj['type']
        time = obj['time']
        key = obj.get('key', "N/A")  # This could be Roshan, tower, etc.
        print(f"Objective: {type_}, Time: {time}, Key: {key}")

# Function to print kills for each player
def print_kills(players):
    print("\n--- Kills by Player ---")
    for player in players:
        hero_id = player['hero_id']
        kills = player['kills_log']  # List of kill events with timestamps
        print(f"Hero ID {hero_id} Kills:")
        for kill in kills:
            time = kill['time']
            print(f"  Kill at {time}s")

# Main function to run the code
def main():
    match_id = "7937741085"  # Replace with your Match ID
    match_data = get_match_data(match_id)
    
    if match_data:
        # Print basic match information
        print("Match ID:", match_id)
        print("Duration:", match_data.get("duration"), "seconds")
        print("Radiant Win:", match_data.get("radiant_win"))

        # Extract and print team fights
        if 'teamfights' in match_data:
            print_team_fights(match_data['teamfights'])
        else:
            print("No team fights data available.")
        
        # Extract and print objectives
        if 'objectives' in match_data:
            print_objectives(match_data['objectives'])
        else:
            print("No objectives data available.")
        
        # Extract and print kills for each player
        if 'players' in match_data:
            print_kills(match_data['players'])
        else:
            print("No players data available.")

# Run the main function
if __name__ == "__main__":
    main()


Match ID: 7937741085
Duration: 4154 seconds
Radiant Win: True

--- Team Fights ---


KeyError: 'damage_done'